I trained the model using a public dataset ,the dataset is part of Sentiment140 student data found at [link](https://docs.google.com/file/d/0B04GJPshIjmPRnZManQwWEdTZjg/edit?resourcekey=0-betyQkEmWZgp8z0DFxWsHw)
I edited the training csv file to create new columns names was not part  of the original and that is what i used to train this model the csv is found at [My drive](https://drive.google.com/file/d/1N4S0xiuP1GnpZcoJiUADQPfg2QE0kigZ/view?usp=sharing)

In [1]:
# Import the necessary libraries
import tensorflow
from keras import regularizers
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, SpatialDropout1D, Bidirectional
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec


nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
# Load the preprocessed dataset
encoding = 'ISO-8859-1'
train_df=pd.read_csv('/content/drive/MyDrive/train.csv', encoding=encoding)

In [ ]:
columns=['polarity','id','date','query','user','text'] 
train_df.columns=columns
train_df

In the sentiment140 dataset tweets have been annotated (0 = negative, 4 = positive) and they can be used to detect sentiment. 

In [ ]:
#Drop the un needed columns
columns=['id','date','query','user']
train_df= train_df.drop(columns,axis=1)
# replace the polarity values to 1 for positive and -1 for negative 
train_df['polarity'] = train_df['polarity'].replace({4: 1, 0: -1})

In [ ]:
#save the changes
train_df.to_csv('/content/drive/MyDrive/train.csv',index=False)

In [ ]:
# shuffle the DataFrame along the rows
train_df = train_df.sample(frac=1, random_state=10000, axis=0)
 # reset the index after shuffling 

train_df.to_csv('/content/drive/MyDrive/train_data.csv',index = False)

In [4]:
train_df

,polarity,text
0,-1,is upset that he can't update his Facebook by ...
1,-1,@Kenichan I dived many times for the ball. Man...
2,-1,my whole body feels itchy and like its on fire
3,-1,"@nationwideclass no, it's not behaving at all...."
4,-1,@Kwesidei not the whole crew
...,...,...
1599994,1,Just woke up. Having no school is the best fee...
1599995,1,TheWDB.com - Very cool to hear old Walt interv...
1599996,1,Are you ready for your MoJo Makeover? Ask me f...
1599997,1,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
#train_df['text'] = train_df['text'].apply(str)
#length =len(train_df)
#train_df = train_df.iloc[:length//2]

In [5]:
# Tokeizing
max_features = 20000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(train_df['text'].values)
train_tokens = tokenizer.texts_to_sequences(train_df['text'].values)
# Pad sequences to make them equal length
train_tokens = pad_sequences(train_tokens)

In [6]:
# convert the sentiment labels to categorical
train_labels = to_categorical(train_df['polarity'], num_classes=2)

In [7]:
train_labels.shape

(1599999, 2)

In [8]:
#build the model
embed_dim = 196
lstm_out = 256

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = train_tokens.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.4, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 117, 196)          3920000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 117, 196)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 256)               463872    
                                                                 
 dense (Dense)               (None, 2)                 514       
                                                                 
Total params: 4,384,386
Trainable params: 4,384,386
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
#split the data for training and testing
X_train, X_test, y_train, y_test = train_test_split(train_tokens, train_labels, test_size=0.2, random_state=42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(1279999, 117) (1279999, 2)
(320000, 117) (320000, 2)


In [ ]:
# Training the model
batch_size = 128
model.fit(X_train, y_train, epochs = 7, batch_size=batch_size, verbose = 1)

Epoch 1/7
  283/10000 [..............................] - ETA: 1:41:28 - loss: 0.0098 - accuracy: 0.9975

In [ ]:
# evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)

# print the evaluation results
print('Test loss:', loss)
print('Test accuracy:', accuracy)


In [ ]:
model.save('/content/drive/MyDrive/model.h5')